In [1]:
import pandas as pd
import os
from pathlib import Path

# Create an empty Df
df = pd.DataFrame()
exp_result_folder = '../../experiment_result/'

# Compute the number of folder in the experiment result folder 
def get_number_of_folders(path):
    return len([name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))])

# Get the number of folders in the experiment result folder
number_of_folders = get_number_of_folders(exp_result_folder)
n_experiments = number_of_folders - 1

parent = Path(exp_result_folder)        # adjust

In [2]:
for i in range(1, n_experiments + 1):        # 1 → n (change to 0-based if you need)
    # “E” + 5-digit zero-padded index, e.g. 1 → E00001, 12 → E00012, 123 → E00123
    pattern = f"E{i:05d}_*"                  # tail “_*” matches any suffix
    for folder in parent.glob(pattern):      # returns Path objects for every match
        print(folder)                        # ↪ or os.chdir(folder) / open files here
        try:
            # any file that ends with “…_experiment_result[.csv]”
            csv_path = next(folder.glob("*_experiment_result*.csv"))
            row_df = pd.read_csv(csv_path, nrows=1)     # one-row DataFrame
            df = pd.concat([df, row_df], ignore_index=True)
        except StopIteration:
            print(f"⚠️ no result file in {folder}")
        except Exception as e:
            print(f"⚠️ error reading {folder}: {e}")

..\..\experiment_result\E00001_250815_ds12_fh1440_m1_naive_hp1


In [3]:
df.tail()

,experiment_no,exp_date,dataset_no,dataset,dataset_freq_min,dataset_length_week,forecast_horizon_min,train_pct,test_pct,model_no,...,hyperparamter,runtime_ms,train_RMSE,train_RMSE_stddev,test_RMSE,test_RMSE_stddev,train_nRMSE,train_nRMSE_stddev,test_nRMSE,test_nRMSE_stddev
270,E00283,2025-07-19,ds13,ashd,30,140,1440,0.9,0.1,m9,...,"seed: 1, max_depth: 15, min_samples_split: 2, ...",1.496552e+03,29.1405,0.992436,74.764500,11.796657,3.774402,0.128545,9.683818,1.527954
271,E00284,2025-07-19,ds13,ashd,30,140,1440,0.9,0.1,m10,...,"seed: 1, n_estimators: 100, max_depth: 3, min_...",5.780129e+05,59.1709,1.323453,60.633700,6.348805,7.664068,0.171419,7.853537,0.822324
272,E00285,2025-07-19,ds13,ashd,30,140,1440,0.9,0.1,m13,...,"seed: 1.0, input_size: 1.0, hidden_size: 64.0,...",2.658953e+06,72.3126,2.396565,77.181300,12.343599,9.366238,0.310413,9.996853,1.598796
273,E00286,2025-07-19,ds13,ashd,30,140,1440,0.9,0.1,m16,...,"seed: 1, seasonality_prior_scale: 10, seasonal...",5.126120e+05,54.5889,5.287208,99.218889,13.852071,7.070588,0.684822,12.851255,1.794180
274,E00287,2025-07-19,ds13,ashd,30,140,1440,0.9,0.1,m17,...,"xgb_seed: 1.0, n_estimators: 200.0, learning_r...",3.034656e+04,17.0866,0.255359,38.755000,6.360114,2.213130,0.033075,5.019714,0.823789


In [3]:
# export df to excel file on experiment_result folder
output_file = Path(exp_result_folder) / 'result_summary.xlsx'
df.to_excel(output_file, sheet_name='result', index=False)